In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 15.0 MB/s eta 0:00:00


In [10]:
!pip install pypdf
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyl

In [1]:
import os
from pypdf import PdfReader
from pypdf.errors import PdfReadError
from tqdm import tqdm
import re

# Define paths
DATA_DIR = "/content/drive/MyDrive/Leo_llm"
OUT_DIR = "/content/drive/MyDrive/Colab Notebooks/data_out"

# Create output directory if it does not exist
os.makedirs(OUT_DIR, exist_ok=True)


PermissionError: [Errno 13] Permission denied: '/content'

In [17]:
def parse_single_doc(file_path):
    text = []
    reader = PdfReader(file_path)

    for page in tqdm(reader.pages[:]):

        # Extract text from the page
        curr_text = page.extract_text()
        if curr_text is None:
            raise ValueError("No text could be extracted from the PDF page.")

        text.append(curr_text)
        text.append("\n\n====================================END PAGE====================================\n\n")

    return "".join(text)

# Function to remove header from text
def remove_header(text):
    header = r'PROCEDURA\s+PRO\d+\s+-P-IT\s+REV\.\s+\d+\s+PRO\d\s+\d+-P-IT\s+REV\.\s+\d+\s+SELEZIONE,\s+AUTORIZZAZIONE\s+E\s+QUALIFICA\s+DEI\s+FORNITORI\s+'
    _RE_HEADER = re.compile(header, re.IGNORECASE)
    text = _RE_HEADER.sub("", text).strip()
    return text

# Function to remove footer from text
def remove_footer(text):
    footer = r'Template:\s+QUA\d+\s+-T-CO\s+it\s+rev\d+\s+©\s+Copyright\s+Selex\s+ES\s+S\.p\.A\.\s+\d+\s+\d+\s+–\s+Tutti\s+i\s+diritti\s+riservati\s+Pag\.\s+\d+\s+di\s+\d+'
    _RE_FOOTER = re.compile(footer, re.IGNORECASE)
    text = _RE_FOOTER.sub("", text).strip()
    return text

# Function to remove empty lines from text
def remove_empty_lines(text):
    _RE_EMPTY_LINES = re.compile(r"^(?:[\t ]*(?:\r?\n|\r))+")
    text = _RE_EMPTY_LINES.sub(" ", text).strip()
    return text


In [18]:
def clean_text_total():
    num_errors = 0

    filenames = sorted(os.listdir(DATA_DIR))
    print(f"Number of documents to process: {len(filenames)}")

    for ind, filename in enumerate(filenames):
        print(f"Processing PDF #{ind + 1}: {filename}")

        try:
            # Construct the full file path
            file_path = os.path.join(DATA_DIR, filename)
            if not os.path.isfile(file_path):
                raise FileNotFoundError(f"The file {file_path} does not exist.")

            # Extract text from PDF
            extracted_text = parse_single_doc(file_path)

            # Clean the extracted text
            cleaned_text = remove_header(extracted_text)
            cleaned_text = remove_footer(cleaned_text)
            cleaned_text = remove_empty_lines(cleaned_text)

            # Write the cleaned text to a new file in OUT_DIR
            output_file_path = os.path.join(OUT_DIR, f"RES_{filename[:12]}.txt")
            with open(output_file_path, "w") as out_file:
                out_file.write(cleaned_text)

            print("Extraction, cleaning, and saving completed successfully!")

        except FileNotFoundError as fnf_error:
            print(f"FileNotFoundError: {fnf_error}")
            num_errors += 1
        except PdfReadError as pdf_error:
            print(f"PdfReadError: Error reading PDF file: {pdf_error}")
            num_errors += 1
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            num_errors += 1

    print(f"Total errors: {num_errors}")
    return cleaned_text


cleaned_text = clean_text_total()


Number of documents to process: 1
Processing PDF #1: PRO002-P-IT rev.01 Selezione  Autorizzazione e Qualifica dei Fornitori cg.pdf


100%|██████████| 42/42 [00:03<00:00, 13.44it/s]


Extraction, cleaning, and saving completed successfully!
Total errors: 0


In [19]:
from langchain.text_splitter import CharacterTextSplitter

class TextChunker:
    def __init__(self, chunk_size=200, size_overlap=0, separator=' '):

        self.chunk_size = chunk_size
        self.size_overlap = size_overlap
        self.separator = separator
        self.text_splitter = CharacterTextSplitter(separator=self.separator, chunk_size=self.chunk_size, chunk_overlap=self.size_overlap)

    def chunk_text(self, text, mode='fixed_size'):

        chunks = self.text_splitter.split_text(text)


        return chunks

    def save_chunks_to_files(self, chunks, output_dir):

        os.makedirs(output_dir, exist_ok=True)

        for i, chunk in enumerate(chunks):
            output_file_path = os.path.join(output_dir, f"chunk_{i+1}.txt")
            with open(output_file_path, "w") as f:
                f.write(chunk)
            print(f"Chunk {i+1} saved to {output_file_path}")


chunker = TextChunker()
chunks = chunker.chunk_text(cleaned_text)
#chunker.save_chunks_to_files(chunks, OUT_DIR)

print(chunks)




Chunk 1 saved to /content/drive/MyDrive/Colab Notebooks/data_out/chunk_1.txt
Chunk 2 saved to /content/drive/MyDrive/Colab Notebooks/data_out/chunk_2.txt
Chunk 3 saved to /content/drive/MyDrive/Colab Notebooks/data_out/chunk_3.txt
Chunk 4 saved to /content/drive/MyDrive/Colab Notebooks/data_out/chunk_4.txt
Chunk 5 saved to /content/drive/MyDrive/Colab Notebooks/data_out/chunk_5.txt
Chunk 6 saved to /content/drive/MyDrive/Colab Notebooks/data_out/chunk_6.txt
Chunk 7 saved to /content/drive/MyDrive/Colab Notebooks/data_out/chunk_7.txt
Chunk 8 saved to /content/drive/MyDrive/Colab Notebooks/data_out/chunk_8.txt
Chunk 9 saved to /content/drive/MyDrive/Colab Notebooks/data_out/chunk_9.txt
Chunk 10 saved to /content/drive/MyDrive/Colab Notebooks/data_out/chunk_10.txt
Chunk 11 saved to /content/drive/MyDrive/Colab Notebooks/data_out/chunk_11.txt
Chunk 12 saved to /content/drive/MyDrive/Colab Notebooks/data_out/chunk_12.txt
Chunk 13 saved to /content/drive/MyDrive/Colab Notebooks/data_out/chun

KeyboardInterrupt: 

In [21]:
for i, chunk in enumerate(chunks):
  print("------ new chunk ---------")
  print(chunk)
  print()

------ new chunk ---------
Le informazioni contenute nel presente documento sono di proprietà di Selex ES S.p.A. e non possono, al pari di tale documento, essere 
riprodott e, utilizzat e o divulgat e in tutto o in parte a

------ new chunk ---------
terzi senza preventiva autorizzazione scritta di Selex ES S.p.A. 
Il documento è disponibile nell’Intranet Aziendale/BMS di Selex ES S.p.A. Le copie, sia in formato elettronico che cartaceo dovranno

------ new chunk ---------
essere verificate, prima dell’utilizzo, con la versione vigente disponibile su Intranet. 
© Copyright Selex ES S.p.A. 2014 - Tutti i diritti riservati 
 
IDENTIFICATIVO : PRO0 02-P-IT Rev. 01 
DATA:

------ new chunk ---------
30/12/2015 
TIPO DOCUMENTO : PROCEDURA 
APPLICAZIONE : Selex ES S.p.A. 
 
 
 
 
 
Selezione, Autorizzazione e Qualifica dei 
Fornitori 
 
 
 
 
 
 
 
 
 
 
 
 
 
SOMMARIO : 
Il presente documento

------ new chunk ---------
descrive le attività di selezione, autorizzazione e qualifica dei forni

In [22]:
from sentence_transformers import SentenceTransformer

class TextEmbedder:
    def __init__(self, encoder_name="BAAI/bge-m3"):

        self.model = SentenceTransformer(encoder_name)

    def generate_embeddings(self, chunks):

        embeddings = self.model.encode(chunks)
        return embeddings


embedder = TextEmbedder()
embeddings = embedder.generate_embeddings(chunks)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

KeyboardInterrupt: 